# Train a classifier to determine product seasonality


First, check if XGBoost is properly installed in the Spark environment (shoud have version 1.0.2)


In [3]:
import pip
display(pip.get_installed_distributions())

Import all necessary libraries.


In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier

from onnxmltools.convert import convert_xgboost
from onnxmltools.convert.common.data_types import FloatTensorType

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Exploratory data analysis (basic stats)

Create Spark temporary views for sales and products.

**IMPORTANT!** Make sure the name of the SQL pool (`SQLPool01` below) matches the name of your SQL pool.


In [7]:
%%spark
val df = spark.read.sqlanalytics("SQLPool01.wwi.SaleSmall") 
df.createOrReplaceTempView("sale")

val df2 = spark.read.sqlanalytics("SQLPool01.wwi.Product") 
df2.createOrReplaceTempView("product")
display(df2)

df: org.apache.spark.sql.DataFrame = [TransactionId: string, CustomerId: int ... 9 more fields]
df2: org.apache.spark.sql.DataFrame = [ProductId: int, Seasonality: int ... 2 more fields]


Load daily product sales from the SQL pool.


In [12]:
sqlQuery = """
SELECT
    P.ProductId
    ,P.Seasonality
    ,S.TransactionDateId
    ,COUNT(*) as TransactionItemsCount
FROM
    sale S
    JOIN product P ON
        S.ProductId = P.ProductId
WHERE
    S.TransactionDateId NOT IN (20120229, 20160229)
GROUP BY
    P.ProductId
    ,P.Seasonality
    ,S.TransactionDateId
"""

prod_df = spark.sql(sqlQuery)
prod_df.cache()

DataFrame[ProductId: int, Seasonality: int, TransactionDateId: int, TransactionItemsCount: bigint]

Check the number of records in the data farame (should be around 13 million rows).

In [7]:
prod_df.count()

13067121

Display some statistics about the data frame.


In [8]:
display(prod_df.describe())

Pivot the data frame to make daily sale items counts columns. 


In [13]:
prod_prep_df = prod_df.groupBy(['ProductId', 'Seasonality']).pivot('TransactionDateId').sum('TransactionItemsCount').toPandas()

/home/trusted-service-user/cluster-env/env/lib/python3.6/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
  warnings.warn("pyarrow.open_stream is deprecated, please use "

Clean up the nulls and take a look at the result.


In [14]:
prod_prep_df = prod_prep_df.fillna(0)
prod_prep_df.head(10)

ProductId  Seasonality  20100102    ...     20191229  20191230  20191231
0        148            2       0.0    ...        447.0    1945.0    1959.0
1        463            1       0.0    ...         33.0     118.0     100.0
2        471            1       1.0    ...         28.0     104.0     110.0
3        496            1       0.0    ...         33.0     114.0      94.0
4        833            1       0.0    ...         31.0     100.0     111.0
5       1088            1       0.0    ...         25.0     104.0     132.0
6       1238            1       0.0    ...         27.0     120.0     107.0
7       1342            1       0.0    ...         31.0     106.0     114.0
8       1580            1       0.0    ...         30.0      77.0     139.0
9       1591            1       0.0    ...         21.0     101.0     135.0

[10 rows x 3650 columns]

Isloate features and prediction classes.

Standardize features by removing the mean and scaling to unit variance.


In [15]:
X = prod_prep_df.iloc[:, 2:].values
y = prod_prep_df['Seasonality'].values

X_scale = StandardScaler().fit_transform(X)

## Use PCA for dimensionality reduction

Perform dimensionality reduction using Principal Components Analysis and two target components.


In [16]:
pca = PCA(n_components=2)
principal_components = pca.fit_transform(X_scale)
principal_components = MinMaxScaler().fit_transform(principal_components)

pca_df = pd.DataFrame(data = principal_components, columns = ['pc1', 'pc2'])
pca_df = pd.concat([pca_df, prod_prep_df[['Seasonality']]], axis = 1)

Display the products data frame in two dimensions (mapped to the two principal components).

Note the clear separation of clusters.


In [17]:
fig = plt.figure(figsize = (6,6))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = [1, 2, 3]
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = pca_df['Seasonality'] == target
    ax.scatter(pca_df.loc[indicesToKeep, 'pc1']
               , pca_df.loc[indicesToKeep, 'pc2']
               , c = color
               , s = 1)
ax.legend(['All Season Products', 'Summer Products', 'Winter Products'])
ax.plot([-0.05, 1.05], [0.77, 1.0], linestyle=':', linewidth=1, color='y')
ax.plot([-0.05, 1.05], [0.37, 0.6], linestyle=':', linewidth=1, color='y')
ax.grid()

plt.show()
plt.close()

Redo the Principal Components Analysis, this time with twenty dimensions.


In [39]:
def col_name(x):
    return f'f{x:02}'

pca = PCA(n_components=20)
principal_components = pca.fit_transform(X_scale)
principal_components = MinMaxScaler().fit_transform(principal_components)

X = pd.DataFrame(data = principal_components, columns = list(map(col_name, np.arange(0, 20))))
pca_df = pd.concat([X, prod_prep_df[['ProductId']]], axis = 1)
pca_automl_df = pd.concat([X, prod_prep_df[['Seasonality']]], axis = 1)

X = X[:4500]
y = prod_prep_df['Seasonality'][:4500]
pca_automl_df = pca_automl_df[:4500]

Save the PCA components to the SQL pool.


In [40]:
pca_sdf = spark.createDataFrame(pca_df)
pca_sdf.createOrReplaceTempView("productpca")

In [41]:
%%spark

val df = spark.sqlContext.sql("select * from productpca")
df.write.sqlanalytics("SQLPool02.wwi_ml.ProductPCA", Constants.INTERNAL)

df: org.apache.spark.sql.DataFrame = [f00: double, f01: double ... 19 more fields]

## Train ensemble of trees classifier (using XGBoost)

Split into test and training data sets.


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

Train the ensemble classifier using XGBoost.


In [24]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=None, subsample=1, tree_method=None,
       validate_parameters=False, verbosity=None)

Perform predictions with the newly trained model.


In [25]:
y_pred = model.predict(X_test)

Calculate the accuracy of the model using test data.


In [26]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 100.00%

Convert trained model to ONNX format.


In [27]:
initial_types = [
    ('input', FloatTensorType([1, 20]))
]

onnx_model = convert_xgboost(model, initial_types=initial_types)

In [124]:

#model.get_booster().get_dump(with_stats=True, dump_format='json')
#model.get_booster().feature_names
#onnx_model

ir_version: 6
producer_name: "OnnxMLTools"
producer_version: "1.5.5"
domain: "onnxconverter-common"
model_version: 0
doc_string: ""
graph {
  node {
    input: "input"
    output: "label"
    output: "probabilities"
    op_type: "TreeEnsembleClassifier"
    attribute {
      name: "class_ids"
      ints: 0
      ints: 0
      ints: 1
      ints: 1
      ints: 2
      ints: 2
      ints: 0
      ints: 0
      ints: 1
      ints: 1
      ints: 2
      ints: 2
      ints: 0
      ints: 0
      ints: 1
      ints: 1
      ints: 2
      ints: 2
      ints: 0
      ints: 0
      ints: 1
      ints: 1
      ints: 2
      ints: 2
      ints: 0
      ints: 0
      ints: 1
      ints: 1
      ints: 2
      ints: 2
      ints: 0
      ints: 0
      ints: 1
      ints: 1
      ints: 2
      ints: 2
      ints: 0
      ints: 0
      ints: 1
      ints: 1
      ints: 2
      ints: 2
      ints: 0
      ints: 0
      ints: 1
      ints: 1
      ints: 2
      ints: 2
      ints: 0
      ints: 0
      

## Train classifier using Auto ML


In [28]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl.run import AutoMLRun
from azureml.train.automl import AutoMLConfig
from azureml.automl.runtime.onnx_convert import OnnxConverter

In [29]:
display(pca_automl_df.head(10))


/opt/spark/python/lib/pyspark.zip/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  'JavaPackage' object is not callable
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.

Configure the connection to the Azure Machine Learning workspace. The Azure portal provides all the values below.


In [30]:
subscription_id='<subscription_id>'         # replace with your Azure subscription id
resource_group='<resource_group_name>'      # replace with your resource group name
workspace_name='<aml_workspace_name>'       # replace with your Azure Machine Learning workspace name
ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
ws.write_config()
ws = Workspace.from_config()
experiment = Experiment(ws, "Product_Seasonality")

Configure the Automated Machine Learning experiment and start it (will run on local compute resources).


In [31]:
automl_classifier_config = AutoMLConfig(
        task='classification',
        #experiment_exit_score = 0.995,
        experiment_timeout_minutes=3,
        enable_onnx_compatible_models=True,
        training_data=pca_automl_df,
        label_column_name='Seasonality',
        n_cross_validations=5,
        enable_voting_ensemble=False,
        enable_stack_ensemble=False
        )

local_run = experiment.submit(automl_classifier_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_0cc2fca5-6418-4d8b-b128-cd7929438fa2

Current status: DatasetCrossValidationSplit. Generating CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Classes are balanced in the training data.

****************************************************************************************************
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
*****************************************************************************************

Retrieve the best model directly in ONNX format and take a look at it.


In [32]:
best_run, onnx_model2 = local_run.get_output(return_onnx_model=True)
#onnx_fl_path = "./best_model.onnx"
#OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

In [34]:
onnx_model2

ir_version: 6
producer_name: "AutoML"
producer_version: "1.0.81"
domain: "ai.onnx"
model_version: 0
doc_string: "{\"AutoMLSDKVer\": \"1.0.81\", \"ExperimentName\": \"Product_Seasonality\", \"RunId\": \"AutoML_0cc2fca5-6418-4d8b-b128-cd7929438fa2_7\", \"PipeId\": \"322fae742ef55d1e4ea31ae1953e1e753c960877\"}"
graph {
  node {
    input: "f00"
    input: "f01"
    input: "f02"
    input: "f03"
    input: "f04"
    input: "f05"
    input: "f06"
    input: "f07"
    input: "f08"
    input: "f09"
    input: "f10"
    input: "f11"
    input: "f12"
    input: "f13"
    input: "f14"
    input: "f15"
    input: "f16"
    input: "f17"
    input: "f18"
    input: "f19"
    output: "variable_all_inputs_concated1"
    name: "FeatureVectorizer"
    op_type: "FeatureVectorizer"
    attribute {
      name: "inputdimensions"
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 

Replace below the placeholders with the name of the primary data lake account and one of it's security keys.

In [33]:
from azure.storage.blob import BlockBlobService

block_blob_service = BlockBlobService(
    account_name='<data_lake_account_name>', account_key='<account_key>') 

block_blob_service.create_blob_from_text('wwi-02', '/ml/onnx/product_seasonality_classifier.onnx', onnx_model2.SerializeToString())